# TPU initialisaton

In [ ]:
try:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address) # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu) 
  # Going back and forth between TPU and host is expensive.
  # Better to run 128 batches on the TPU before reporting back.
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
  print("Number of accelerators: ", strategy.num_replicas_in_sync)
except ValueError:
  print('TPU failed to initialize.')

In [2]:
import tensorflow as tf
import random

In [3]:
AUTO = tf.data.experimental.AUTOTUNE

HEIGHT=720
WIDTH=1280
COLORS=3

# Reading dataset from tf records

We start with exploring the images, where we see at least one instance of image

In [4]:
!cp -r "/content/drive/MyDrive/deep learning/Object_detection/Coral_Reef/tf_records_data/seen_data" "/home/"

In [14]:
DATA_PATH= '/home/seen_data/seen_data*'
BATCH_SIZE = 64

VALIDATION_SPLIT = 0.2
filenames = tf.io.gfile.glob(DATA_PATH)
random.shuffle(filenames)

split = int(len(filenames) * VALIDATION_SPLIT)
training_filenames = filenames[split:]
validation_filenames = filenames[:split]

print(f"Splitting dataset into {len(training_filenames)} training files and {len(validation_filenames)} validation files")

Splitting dataset into 13 training files and 3 validation files


In [6]:
feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'width': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'detections_number':tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'image': tf.io.FixedLenFeature([], tf.string),
    'path': tf.io.VarLenFeature(tf.string),
    'sequence_id':tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'video_id':tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'video_frame':tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'sequence_frame':tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'bbox/xmin':tf.io.VarLenFeature(tf.float32),
    'bbox/xmax':tf.io.VarLenFeature(tf.float32),
    'bbox/ymin':tf.io.VarLenFeature(tf.float32),
    'bbox/ymax':tf.io.VarLenFeature(tf.float32),
    'class/text': tf.io.VarLenFeature(tf.string),
    'class/label': tf.io.VarLenFeature(tf.int64)
}

In [176]:
def read_tfrecord(example,feature_description=feature_description):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  x = tf.io.parse_single_example(example, feature_description)

  image= tf.io.decode_jpeg(x["image"], channels=3)
  #image=tf.cast(image,tf.float32)/255.0
  x_mins=tf.sparse.to_dense(x['bbox/xmin'])
  y_mins=tf.sparse.to_dense(x['bbox/ymin'])
  x_maxs=tf.sparse.to_dense(x['bbox/xmax'])
  y_maxs=tf.sparse.to_dense(x['bbox/ymax'])
  #n_occurances=x['detections_number']
  return image,tf.Tensor((x_mins,y_mins,x_maxs,y_maxs),dtype=tf.uint8,value_index=1)

In [177]:
def load_dataset(filenames):
  # read from TFRecords. For optimal performance, use "interleave(tf.data.TFRecordDataset, ...)"
  # to read from multiple TFRecord files at once and set the option experimental_deterministic = False
  # to allow order-altering optimizations.

  option_no_order = tf.data.Options()
  option_no_order.experimental_deterministic = False

  dataset = tf.data.Dataset.from_tensor_slices(filenames)
  dataset = dataset.with_options(option_no_order)
  dataset = dataset.interleave(tf.data.TFRecordDataset, cycle_length=16, num_parallel_calls=AUTO) # faster
  dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
  return dataset

def get_batched_dataset(filenames):
  dataset = load_dataset(filenames)
  dataset = dataset.shuffle(100)
  dataset = dataset.batch(BATCH_SIZE, drop_remainder=False) # drop_remainder will be needed on TPU
  dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
  return dataset

def get_training_dataset():
  dataset = get_batched_dataset(training_filenames)
  dataset = strategy.experimental_distribute_dataset(dataset)
  return dataset

def get_validation_dataset():
  dataset = get_batched_dataset(validation_filenames)
  dataset = strategy.experimental_distribute_dataset(dataset)
  return dataset

In [178]:
dataset = load_dataset(filenames)

for i,x in enumerate(load_dataset(filenames)):
  print(x[1].shape)
  if i==300:
    break


TypeError: ignored

In [149]:
dataset=dataset.shuffle(100)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
dataset = dataset.prefetch(AUTO)

In [154]:
for x in dataset:
  break

In [131]:
v=next(iter(get_batched_dataset(filenames)))

InvalidArgumentError: ignored

# TPU initializtion

#Importing object detection API

In [22]:
!cd /home/barrier_reef
!rm -rf ./models/

!git clone --depth 1 https://github.com/tensorflow/models/
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

/bin/bash: line 0: cd: /home/barrier_reef: No such file or directory
Cloning into 'models'...
remote: Enumerating objects: 3203, done.
remote: Counting objects: 100% (3203/3203), done.
remote: Compressing objects: 100% (2717/2717), done.
remote: Total 3203 (delta 847), reused 1354 (delta 442), pack-reused 0
Receiving objects: 100% (3203/3203), 33.41 MiB | 25.84 MiB/s, done.
Resolving deltas: 100% (847/847), done.


In [23]:
%%writefile models/research/setup.py

import os
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = [
    'tf-models-official==2.7.0',
    'tensorflow_io'
]

setup(
    name='object_detection',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    include_package_data=True,
    packages=(
        [p for p in find_packages() if p.startswith('object_detection')] +
        find_packages(where=os.path.join('.', 'slim'))),
    package_dir={
        'datasets': os.path.join('slim', 'datasets'),
        'nets': os.path.join('slim', 'nets'),
        'preprocessing': os.path.join('slim', 'preprocessing'),
        'deployment': os.path.join('slim', 'deployment'),
        'scripts': os.path.join('slim', 'scripts'),
    },
    description='Tensorflow Object Detection Library',
    python_requires='>3.6',
)


Writing models/research/setup.py


In [24]:
!python -m pip install models/research

Processing ./models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 23.1 MB 2.1 MB/s 
     |████████████████████████████████| 90 kB 7.8 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 99 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 36.9 MB/s 
     |████████████████████████████████| 213 kB 40.2 MB/s 
     |████████████████████████████████| 4.9 MB 38.6 MB/s 
     |████████████████████████████████| 47.6 MB 56 kB/s 
     |████████████████████████████████| 1.2 MB 43.0 MB/s 
   

# Downloading the model

The list of models can be found in https://github.com/tensorflow/models

Size of inputs 1536x1536

In [25]:
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

from object_detection.utils import colab_utils

In [35]:
model_name="efficientdet_d7_coco17_tpu-32"

In [36]:
! echo {model_name}

efficientdet_d7_coco17_tpu-32


In [37]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/{model_name}.tar.gz
!tar -xf {model_name}.tar.gz
!mv {model_name}/checkpoint models/research/object_detection/test_data/

--2021-12-19 14:48:44--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d7_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.12.128, 2607:f8b0:400c:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.12.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394474998 (376M) [application/x-tar]
Saving to: ‘efficientdet_d7_coco17_tpu-32.tar.gz’

efficientdet_d7_coc 100%[===================>] 376.20M   121MB/s    in 3.1s    

2021-12-19 14:48:47 (121 MB/s) - ‘efficientdet_d7_coco17_tpu-32.tar.gz’ saved [394474998/394474998]



Configuring the model

In [41]:
tf.keras.backend.clear_session()

pipeline_config = f'models/research/object_detection/configs/tf2/ssd_efficientdet_d7_1536x1536_coco17_tpu-32.config'
checkpoint_path = 'models/research/object_detection/test_data/checkpoint/ckpt-0'

configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']

{'eval_config': metrics_set: "coco_detection_metrics"
 use_moving_averages: false
 batch_size: 1,
 'eval_input_config': label_map_path: "PATH_TO_BE_CONFIGURED/label_map.txt"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BEE_CONFIGURED/val2017-?????-of-00032.tfrecord"
 },
 'eval_input_configs': [label_map_path: "PATH_TO_BE_CONFIGURED/label_map.txt"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BEE_CONFIGURED/val2017-?????-of-00032.tfrecord"
 }
 ],
 'model': ssd {
   num_classes: 90
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 1536
       max_dimension: 1536
       pad_to_max_dimension: true
     }
   }
   feature_extractor {
     type: "ssd_efficientnet-b6_bifpn_keras"
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
         

In [45]:
# Modifying the configuration of the model

model_config.ssd.num_classes = 1
model_config.ssd.freeze_batchnorm = True

#Building modified model
with strategy.scope():
  detection_model = model_builder.build(model_config=model_config,
                                        is_training=True)

#Restoring the weights for detection model



#Restoring the weights for detection model

We do not want to use all of the efficient net configuration, as we have only one class, we want to use the feature extraction layer and the bounding box regression prediction layer and do not include the class prediction layer

In [49]:
tmp_box_predictor_checkpoint  = tf.train.Checkpoint(
    _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
    _box_prediction_head=detection_model._box_predictor._box_prediction_head,
)

tmp_model_checkpoint = tf.train.Checkpoint(
    _feature_extractor=detection_model._feature_extractor,
    _box_predictor=tmp_box_predictor_checkpoint 
)

checkpoint = tf.train.Checkpoint(model=tmp_model_checkpoint)
checkpoint.restore(checkpoint_path).expect_partial()

In [50]:
#Runnig dummy images to restore weights
tmp_image, tmp_shapes =  detection_model.preprocess(tf.zeros([1, 1536, 1536, 3]))
tmp_prediction_dict = detection_model.predict(tmp_image, tmp_shapes)
tmp_detections = detection_model.postprocess(tmp_prediction_dict, tmp_shapes)

print('Weights restored!')

Weights restored!


In [ ]:
for i,v in enumerate(detection_model.trainable_variables):
    print(f"i: {i} \t name: {v.name} \t shape:{v.shape} \t dtype={v.dtype}")

In [72]:
to_fine_tune = [v for i,v in enumerate(detection_model.trainable_variables)
                if v.name.startswith(r'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead/BoxPredictor') |
                   v.name.startswith(r'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead/ClassPredictor') |
                   ((i >6) & (i<45)) ]

# Provide dataset for training

In [77]:
x[1]

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([114.], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([402.], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([154.], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([429.], dtype=float32)>)

In [106]:
x[0]

<tf.Tensor: shape=(720, 1280, 3), dtype=float32, numpy=
array([[[0.        , 0.4745098 , 0.8666667 ],
        [0.        , 0.47843137, 0.87058824],
        [0.        , 0.47843137, 0.8627451 ],
        ...,
        [0.02745098, 0.24313726, 0.3254902 ],
        [0.01960784, 0.23529412, 0.31764707],
        [0.02352941, 0.23921569, 0.32156864]],

       [[0.        , 0.4745098 , 0.8666667 ],
        [0.        , 0.47843137, 0.87058824],
        [0.        , 0.47843137, 0.8627451 ],
        ...,
        [0.02352941, 0.23921569, 0.32156864],
        [0.01960784, 0.23529412, 0.31764707],
        [0.02352941, 0.23921569, 0.32156864]],

       [[0.        , 0.4745098 , 0.8666667 ],
        [0.        , 0.47843137, 0.87058824],
        [0.        , 0.47843137, 0.8627451 ],
        ...,
        [0.02352941, 0.23921569, 0.32156864],
        [0.01960784, 0.23529412, 0.31764707],
        [0.02352941, 0.23921569, 0.32156864]],

       ...,

       [[0.20784314, 0.77254903, 0.77254903],
        [0.1

In [115]:
tf.Variable([1,x[0]])

InvalidArgumentError: ignored

In [121]:
x[0]

<tf.Tensor: shape=(720, 1280, 3), dtype=float32, numpy=
array([[[0.07450981, 0.46666667, 0.56078434],
        [0.08627451, 0.47843137, 0.5803922 ],
        [0.07058824, 0.45882353, 0.57254905],
        ...,
        [0.        , 0.5568628 , 0.99607843],
        [0.        , 0.5568628 , 0.9882353 ],
        [0.        , 0.5568628 , 0.9882353 ]],

       [[0.08235294, 0.47843137, 0.5882353 ],
        [0.09019608, 0.4862745 , 0.6039216 ],
        [0.07058824, 0.4745098 , 0.59607846],
        ...,
        [0.        , 0.5529412 , 0.99215686],
        [0.        , 0.5529412 , 0.9843137 ],
        [0.        , 0.5529412 , 0.9843137 ]],

       [[0.09411765, 0.49411765, 0.6431373 ],
        [0.09411765, 0.5019608 , 0.64705884],
        [0.09019608, 0.5137255 , 0.6627451 ],
        ...,
        [0.        , 0.5529412 , 0.99215686],
        [0.        , 0.5529412 , 0.9843137 ],
        [0.        , 0.5529412 , 0.9843137 ]],

       ...,

       [[0.8745098 , 0.9647059 , 0.9882353 ],
        [0.8

In [ ]:
get_batched_dataset(training_filenames)

In [123]:
print('Start fine-tuning!', flush=True)

for x in get_batched_dataset(training_filenames):
  print(j)
  break
  gt_boxes_list = [[x[1][0][i],x[1][1][i],x[1][2][i],x[1][3][i]] for i in range(len(x[1][0]))]
  gt_classes=[1]*len(x[1][0])
  
  x['image']


Start fine-tuning!


InvalidArgumentError: ignored

In [90]:
[i for i in range(4)]

[0, 1, 2, 3]

In [ ]:
image=tf.image.resize(image,(1536,1536,3))

In [ ]:
@tf.function
def train_step_fn(image_list,
                  groundtruth_boxes_list,
                  groundtruth_classes_list,
                  model,
                  optimizer,
                  vars_to_fine_tune):
    """A single training iteration.

    Args:
      image_list: A list of [1, height, width, 3] Tensor of type tf.float32.
        Note that the height and width can vary across images, as they are
        reshaped within this function to be 640x640.
      groundtruth_boxes_list: A list of Tensors of shape [N_i, 4] with type
        tf.float32 representing groundtruth boxes for each image in the batch.
      groundtruth_classes_list: A list of Tensors of shape [N_i, num_classes]
        with type tf.float32 representing groundtruth boxes for each image in
        the batch.

    Returns:
      A scalar tensor representing the total loss for the input batch.
    """
    tf.keras.backend.set_learning_phase(True)
  
    shapes = tf.constant(batch_size * [[1530, 1530, 3]], dtype=tf.int32)
    model.provide_groundtruth(
        groundtruth_boxes_list=groundtruth_boxes_list,
        groundtruth_classes_list=groundtruth_classes_list)
    
    with tf.GradientTape() as tape:
    ### START CODE HERE (Replace instances of `None` with your code) ###

        # Preprocess the images
        
        preprocessed_image_tensor  = tf.concat(
           [detection_model.preprocess(image_tensor)[0]
           for image_tensor in image_list], axis=0)
        true_shape_tensor =  shapes

        # Make a prediction
        prediction_dict = model.predict(preprocessed_image_tensor, true_shape_tensor)

        # Calculate the total loss (sum of both losses)  
        losses_dict= model.loss(prediction_dict, true_shape_tensor)
    
        total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']

        # Calculate the gradients
        gradients = tape.gradient(total_loss,vars_to_fine_tune)

        # Optimize the model's selected variables
        vars_to_fine_tune=optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))

        ### END CODE HERE ###
        
    return total_loss

In [74]:
import pandas as pd
from pathlib import Path
from math import ceil
import json

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import zipfile
import io
import scipy.misc
import numpy as np

import glob
import imageio
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

from object_detection.utils import colab_utils

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path.

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)


def draw_image_with_boxes(image_number,test=None):
  if test==None:
    vid_id,frame_num=train_df.loc[image_number,['video_id','video_frame']].values
  else:
    vid_id,frame_num=test_df.loc[image_number,['video_id','video_frame']].values

  path_to_image=path_to_data/'train_images'/f"video_{vid_id}"/f"{frame_num}.jpg"

  box_dict=train_df.loc[image_number,'annotations']
  box_dict=json.loads(box_dict.replace("'", '"'))
  image = Image.open(path_to_image)
  
  for v in (box_dict):
    x_min=v['x']
    x_max=v['x']+v['width']
    y_min=v['y']
    y_max=v['y']+v['height']
    
    viz_utils.draw_bounding_box_on_image(image,
                                        y_min,
                                        x_min,
                                        y_max,
                                        x_max,
                                        use_normalized_coordinates=False)
  
  return image

In [ ]:
test_share=0.25
data=pd.read_csv(path_to_data/'train.csv')

index_values=data.groupby('video_id').apply(lambda x: max(x.index)).rename('last_index').reset_index()
index_values['first_index']=(index_values['last_index']*(1-test_share)+ \
                             index_values['last_index'].shift(1).fillna(0)*test_share).apply(ceil)


test_filter=np.zeros(data.shape[0])

for _,val in index_values.iterrows():
  test_filter[val['first_index']:val['last_index']+1]=1

test_df=data[test_filter==1]
train_df=data[test_filter!=1]

train_df=(train_df[train_df['annotations']!='[]']).reset_index(drop=True)
test_df=(test_df[test_df['annotations']!='[]']).reset_index(drop=True)

print(f"Number of items in train dataset {train_df.shape[0]}")
print(f"Number of items in test dataset {test_df.shape[0]}")

In [ ]:
%matplotlib inline

In [ ]:
draw_image_with_boxes(1667,test=None)

In [ ]:
draw_image_with_boxes(210,test=None)

In [ ]:
draw_image_with_boxes(1450,test=None)

In [ ]:
with strategy.scope():